# Creating Lamda Function

In this notebook, we are going to create a Lamda Function to invoke the endpoint created by the CI/CD pipeline.

## The Steps Involved in this process are :-


>- Step 1 : Create an iam role for Lambda and attach necessary policies to it
>- Step 2 : Build OpenCV library using Docker
>- Step 3 : Deploy - Lambda function with libraries as Lambda layers


---

## Step 1: Create an iam role for Lambda and attach necessary policies to it

In [3]:
import json
import os
import boto3
from botocore.exceptions import ClientError


# Create IAM client
iam_client = boto3.client('iam')

# Create a policy
trust_relationship_policy_another_iam_user = {"Version":"2012-10-17",
                     "Statement":{
                         "Effect":"Allow",
                          "Principal":{
                              "Service":
                               "lambda.amazonaws.com"
                              },
                          "Action":"sts:AssumeRole"
                         }
                    }

role_name = "openCV-Lambda"
try:
    create_role_response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_relationship_policy_another_iam_user),
        Description='This is a role with S3 and Lambda exec access',
    )
except ClientError as error:
    if error.response['Error']['Code'] == 'EntityAlreadyExists':
        print('Role already exists... hence exiting from here')
    else:
        print('Unexpected error occurred... Role could not be created', error)

Role already exists... hence exiting from here


---

Attach AmazonS3FullAccess,AWSLambdaBasicExecutionRole and AmazonSageMakerFullAccess policies to the lambda

In [4]:
policy_json = {
    "Version": "2012-10-17",
    "Statement": [{
        "Effect": "Allow",
        "Action": [
            "ec2:*"
        ],
        "Resource": "*"
    }]
}

policy_name = role_name + '_policy'
policy_arns = ['arn:aws:iam::aws:policy/AmazonS3FullAccess','arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole',
              'arn:aws:iam::aws:policy/AmazonSageMakerFullAccess']

for policy_arn in policy_arns:
    try:
        policy_attach_res = iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=policy_arn
        )
    except ClientError as error:
        print('Unexpected error occurred... hence cleaning up')
        iam_client.delete_role(
            RoleName= role_name
        )
        print('Role could not be created...', error)

---

## Step 2:  Build OpenCV library using Docker

In [24]:
!git clone https://github.com/iandow/opencv_aws_lambda
os.chdir("opencv_aws_lambda")
os.getcwd()
!docker build --tag=lambda-layer-factory:latest .
!docker run --rm -it -v $(pwd):/data lambda-layer-factory cp /packages/cv2-python37.zip /data

Cloning into 'opencv_aws_lambda'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 123 (delta 3), reused 6 (delta 2), pack-reused 112
Receiving objects: 100% (123/123), 42.71 MiB | 14.27 MiB/s, done.
Resolving deltas: 100% (61/61), done.
Sending build context to Docker daemon  45.49MB
Step 1/39 : FROM amazonlinux
 ---> 24608685b402
Step 2/39 : RUN yum update -y
 ---> Using cache
 ---> 4d0f91c18bc8
Step 3/39 : RUN yum install gcc openssl-devel bzip2-devel libffi-devel wget tar gzip zip make -y
 ---> Using cache
 ---> da982985bd3c
Step 4/39 : WORKDIR /
 ---> Using cache
 ---> 0c55b228622c
Step 5/39 : RUN wget https://www.python.org/ftp/python/3.9.10/Python-3.9.10.tgz
 ---> Using cache
 ---> 4b911aa480c7
Step 6/39 : RUN tar -xzvf Python-3.9.10.tgz
 ---> Using cache
 ---> 8c73bea6e58a
Step 7/39 : WORKDIR /Python-3.9.10
 ---> Using cache
 ---> a4fe9608b8f4
Step 8/39 : RUN ./configure --enable

gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Objects/cellobject.o Objects/cellobject.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Objects/classobject.o Objects/classobject.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Objects/codeobject.o Objects/codeobjec

gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Objects/picklebufobject.o Objects/picklebufobject.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Objects/rangeobject.o Objects/rangeobject.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Objects/setobject.o Objects/

gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Python/getargs.o Python/getargs.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Python/getcompiler.o Python/getcompiler.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Python/getcopyright.o Python/getcopyright.c
gcc 

gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Python/pythonrun.o Python/pythonrun.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Python/pytime.o Python/pytime.c
gcc -pthread -c -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall    -std=c99 -Wextra -Wno-unused-result -Wno-unused-parameter -Wno-missing-field-initializers -Werror=implicit-function-declaration -fvisibility=hidden  -I./Include/internal  -I. -I./Include    -DPy_BUILD_CORE -o Python/bootstrap_hash.o Python/bootstrap_hash.c
gcc -p

...

  adding: python/lib/python3.9/site-packages/numpy/core/include/numpy/libdivide/libdivide.h (deflated 84%)
  adding: python/lib/python3.9/site-packages/numpy/core/include/numpy/libdivide/LICENSE.txt (deflated 48%)
  adding: python/lib/python3.9/site-packages/numpy/core/include/numpy/npy_3kcompat.h (deflated 74%)
  adding: python/lib/python3.9/site-packages/numpy/core/include/numpy/npy_math.h (deflated 81%)
  adding: python/lib/python3.9/site-packages/numpy/core/include/numpy/old_defines.h (deflated 79%)
  adding: python/lib/python3.9/site-packages/numpy/core/include/numpy/npy_endian.h (deflated 79%)
  adding: python/lib/python3.9/site-packages/numpy/core/generate_numpy_api.py (deflated 69%)
  adding: python/lib/python3.9/site-packages/numpy/core/numerictypes.pyi (deflated 69%)
  adding: python/lib/python3.9/site-packages/numpy/core/_add_newdocs.py (deflated 75%)
  adding: python/lib/python3.9/site-packages/numpy/core/shape_base.py (deflated 73%)
  adding: python/lib/python3.9/site-pack

  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_array_coercion.cpython-39.pyc (deflated 63%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_shape_base.cpython-39.pyc (deflated 66%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_overrides.cpython-39.pyc (deflated 68%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_records.cpython-39.pyc (deflated 60%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_array_interface.cpython-39.pyc (deflated 58%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_dtype.cpython-39.pyc (deflated 65%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_function_base.cpython-39.pyc (deflated 65%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/__pycache__/test_multiarray.cpython-39.pyc (deflated 66%)
  adding: python/lib/python3.9/site-p

  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_multiarray.py (deflated 80%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test__exceptions.py (deflated 68%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_numeric.py (deflated 81%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_api.py (deflated 78%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_cython.py (deflated 68%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_array_coercion.py (deflated 74%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_limited_api.py (deflated 52%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_arrayprint.py (deflated 80%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/test_longdouble.py (deflated 81%)
  adding: python/lib/python3.9/site-packages/numpy/core/tests/_locales.py (deflated 64%)
  adding: python/lib/python3.9/site-packages/numpy/core

  adding: python/lib/python3.9/site-packages/numpy/core/_machar.py (deflated 73%)
  adding: python/lib/python3.9/site-packages/numpy/core/_exceptions.py (deflated 69%)
  adding: python/lib/python3.9/site-packages/numpy/core/__init__.py (deflated 58%)
  adding: python/lib/python3.9/site-packages/numpy/core/getlimits.pyi (deflated 21%)
  adding: python/lib/python3.9/site-packages/numpy/core/_internal.pyi (deflated 59%)
  adding: python/lib/python3.9/site-packages/numpy/core/_multiarray_umath.cpython-39-x86_64-linux-gnu.so (deflated 69%)
  adding: python/lib/python3.9/site-packages/numpy/core/setup_common.py (deflated 68%)
  adding: python/lib/python3.9/site-packages/numpy/core/einsumfunc.py (deflated 73%)
  adding: python/lib/python3.9/site-packages/numpy/_version.py (deflated 30%)
  adding: python/lib/python3.9/site-packages/numpy/_pytesttester.py (deflated 63%)
  adding: python/lib/python3.9/site-packages/numpy/__init__.py (deflated 62%)
  adding: python/lib/python3.9/site-packages/num

  adding: python/lib/python3.9/site-packages/cv2/cv2.abi3.so (deflated 59%)
  adding: python/lib/python3.9/site-packages/cv2/qt/ (stored 0%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/ (stored 0%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSansCondensed-Bold.ttf (deflated 51%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSans-BoldOblique.ttf (deflated 49%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSans-Oblique.ttf (deflated 49%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSans.ttf (deflated 50%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSansCondensed.ttf (deflated 51%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSansCondensed-Oblique.ttf (deflated 48%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSansCondensed-BoldOblique.ttf (deflated 48%)
  adding: python/lib/python3.9/site-packages/cv2/qt/fonts/DejaVuSans-Bold.ttf (deflated 50%

---

## Step 3: Deploy  - Lambda function with libraries as Lambda layers.

### 1. Edit the Lambda function code according to your requirement

In [26]:
%%writefile app.py

import json
import pickle
import os
import boto3
import numpy as np
import cv2
import boto3
import io

def split_s3_path(s3_path):
    path_parts=s3_path.replace("s3://","").split("/")
    bucket=path_parts.pop(0)
    key="/".join(path_parts)
    return bucket, key

def lambda_handler(event, context):
    # TODO implement
    
    print(event)
    ENDPOINT_NAME = os.environ["ENDPOINT_NAME"]
    request_args = {}
    body = event
    path = json.loads(json.dumps(event))
    path = path["s3_path"]
    print("path",path) 
    s3 = boto3.resource('s3', region_name='ap-south-1')
    bucket, key = split_s3_path(path)

    bucket = s3.Bucket(bucket)
    object = bucket.Object(key)
    response = object.get()['Body'].read()
    file_stream =response
    body = bytearray(file_stream)


    request_args['Body'] = body
    request_args['EndpointName'] = ENDPOINT_NAME
    request_args['ContentType'] = 'application/x-image'
    request_args['Accept'] = 'application/json'
 
    # # works successfully
    runtime= boto3.client('runtime.sagemaker')
    
    response = runtime.invoke_endpoint(**request_args)
    
    response_body = response['Body'].read()
    detections = json.loads(response_body) 
    
    return {
        'statusCode': 200,
        'body': json.dumps(detections)
    }


Overwriting app.py


### 2. Publish the OpenCV Python library as a Lambda layer.

In [27]:
ACCOUNT_ID = boto3.client("sts").get_caller_identity()["Account"]

LAMBDA_LAYERS_BUCKET = f"lambda-layers-{ACCOUNT_ID}"
LAYER_NAME="cv2"
!aws s3 mb s3://$LAMBDA_LAYERS_BUCKET
!aws s3 cp cv2-python37.zip s3://$LAMBDA_LAYERS_BUCKET
!aws lambda publish-layer-version --layer-name $LAYER_NAME --description "Open CV" --content S3Bucket=$LAMBDA_LAYERS_BUCKET,S3Key=cv2-python37.zip --compatible-runtimes python3.7

make_bucket failed: s3://lambda-layers-296512243111 An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.
upload: ./cv2-python37.zip to s3://lambda-layers-296512243111/cv2-python37.zip
{
    "Content": {
        "Location": "https://awslambda-ap-s-1-layers.s3.ap-south-1.amazonaws.com/snapshots/296512243111/cv2-3ea78e62-24e5-4653-b3d7-e90e4edcab86?versionId=TEI9grJcETERFK6jN2Qd1A8uRM2I1Vwz&X-Amz-Security-Token=IQoJb3JpZ2luX2VjENn%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCmFwLXNvdXRoLTEiRzBFAiAj7ziNpUwTa7LshdwnSYUFjw8g7w5UaKYTWDbS3Y%2BjcgIhAOqkNLZbBhXh0H4jgz9q5wVBe%2FFNPphaC35qIldRmNgnKtkECNL%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEQAxoMNTQ1MjU1MjAxMzA3Igwmo9z7C3z2KY0evsQqrQTIMOgLIgvbww0%2BnJ5%2BuGdfpraGBZrQGiZgeS7pBgHKTVTk5W7cEJKjQVVkgh9vUpUrbW6j4u%2FrjEn1ofHTlHUmW0JpQ7%2BS2PXLGMnL1z1Lv9gQWOp7Nayh606EfyoTBxemSZe9xt4nNbYy5GtkA6B%2BVol2jn4BRHSwJVxNanEPpxevZzOBhK%2FjXjly6hw7K6qI8dsrGTf6HRG5uO2jR

### 3. Create and Deploy the Lambda function:

In [28]:
!zip app.zip app.py

  adding: app.py (deflated 53%)


In [29]:
import sagemaker
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

BUCKET_NAME = bucket


In [30]:
FUNCTION_NAME="openCV_Layered"
ACCOUNT_ID= boto3.client("sts").get_caller_identity()["Account"]
!aws s3 cp app.zip s3://$BUCKET_NAME

upload: ./app.zip to s3://sagemaker-ap-south-1-296512243111/app.zip


---

Loading the endpoint-name

In [32]:
endpoint_name = "<<Sagemaker Endpoint Name>>" # Visit AWS management console →Amazon Sagemaker →Inference →Endpoints to get the endpoin_name

In [35]:
lambda_client = boto3.client('lambda')
response = lambda_client.create_function(
    FunctionName=FUNCTION_NAME,
    Runtime='python3.7',
    Role=f"arn:aws:iam::{ACCOUNT_ID}:role/{role_name}",
    Handler='app.lambda_handler',
    Code={
        'S3Bucket': BUCKET_NAME,
        'S3Key': 'app.zip',
    },
    Timeout=600,
    MemorySize = 10240,
    PackageType='Zip',
    Environment={
        'Variables': {
            'ENDPOINT_NAME': "pytorch-training-2022-11-14-07-41-12-045"
        }
    },
)

### 4. Attach the cv2 Lambda layer to our Lambda function:

In [36]:
response = lambda_client.list_layer_versions(
    LayerName=LAYER_NAME,
)
LAYER = response['LayerVersions'][0]['LayerVersionArn']
LAYER

'arn:aws:lambda:ap-south-1:296512243111:layer:cv2:13'

In [37]:
response

{'ResponseMetadata': {'RequestId': '553617e1-e5f2-472f-83c6-34b696da1ae2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 14 Nov 2022 09:54:00 GMT',
   'content-type': 'application/json',
   'content-length': '1240',
   'connection': 'keep-alive',
   'x-amzn-requestid': '553617e1-e5f2-472f-83c6-34b696da1ae2'},
  'RetryAttempts': 0},
 'LayerVersions': [{'LayerVersionArn': 'arn:aws:lambda:ap-south-1:296512243111:layer:cv2:13',
   'Version': 13,
   'Description': 'Open CV',
   'CreatedDate': '2022-11-14T09:52:49.271+0000',
   'CompatibleRuntimes': ['python3.7']},
  {'LayerVersionArn': 'arn:aws:lambda:ap-south-1:296512243111:layer:cv2:12',
   'Version': 12,
   'Description': 'Open CV',
   'CreatedDate': '2022-11-14T09:23:33.013+0000',
   'CompatibleRuntimes': ['python3.7']},
  {'LayerVersionArn': 'arn:aws:lambda:ap-south-1:296512243111:layer:cv2:11',
   'Version': 11,
   'Description': 'Open CV',
   'CreatedDate': '2021-09-22T05:30:38.078+0000',
   'CompatibleRuntimes': ['python3

In [38]:
response = lambda_client.update_function_configuration(
    FunctionName=FUNCTION_NAME,
    Layers=[LAYER],
    )

---

# Now you can deploy the enpoint via API Gateway

>- Follow this detailed [AWS tutorial]("https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/") to invoke lambda function via Amazon API gateway 


